In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
# from utils.df_handle import *
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='SOD'
prefix='SYNC_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '*/30 8-17,23-23 * * *',
          tags=[prefix+name, 'Sync', '30mins']
)

In [4]:
start_date = '2022-01-01'
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y-%m-%d")

In [5]:
from_tb = "OM_SalesOrdDet"

In [11]:
sql = \
f"""
DECLARE @from DATE = '{start_date}'
DECLARE @to DATE = '2022-01-31'
SELECT
CONCAT(BranchID, OrderNbr, LineRef) as pk,
BranchID,
OrderNbr,
LineRef,
FreeItem,
InvtID,
LineQty,
OrderType,
OrigOrderNbr,
SiteID,
Crtd_Prog,
Crtd_User,
Crtd_Datetime,
LUpd_Datetime,
SlsperID,
OriginalLineRef,
BeforeVATPrice,
BeforeVATAmount,
AfterVATPrice,
AfterVATAmount,
VATAmount
from {from_tb}
where Crtd_Datetime >= @from
"""

In [12]:
table_name = "sync_dms_sod1"
table_temp = "sync_dms_sod_temp"

In [10]:
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
bq_values_insert(df, f"{table_temp}", 3)
sql = \
f"""
DROP TABLE IF EXISTS biteam.{table_name};
CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
PARTITION BY DATE(crtd_datetime)
CLUSTER BY pk,branchid,origordernbr,ordernbr
"""
execute_bq_query(sql)

1it [00:06,  6.69s/it]


In [13]:
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
bq_values_insert(df, f"{table_name}", 2)

1it [00:28, 28.11s/it]


In [1]:
usql = \
f"""
DECLARE @from DATE = '{datenow}'
DECLARE @to DATE = '2022-01-31'
SELECT
CONCAT(BranchID, OrderNbr, LineRef) as pk,
BranchID,
OrderNbr,
LineRef,
FreeItem,
InvtID,
LineQty,
OrderType,
OrigOrderNbr,
SiteID,
Crtd_Prog,
Crtd_User,
Crtd_Datetime,
LUpd_Datetime,
SlsperID,
OriginalLineRef,
BeforeVATPrice,
BeforeVATAmount,
AfterVATPrice,
AfterVATAmount,
VATAmount
from {from_tb}
where LUpd_DateTime >= @from
and Crtd_DateTime < @from
"""

NameError: name 'datenow' is not defined

In [50]:
print(usql)


DECLARE @from DATE = '2022-05-14'
DECLARE @to DATE = '2022-01-31'
SELECT
BranchID,
OrderNbr,
LineRef,
FreeItem,
InvtID,
LineQty,
OrderType,
OrigOrderNbr,
SiteID,
Crtd_Prog,
Crtd_User,
Crtd_Datetime,
LUpd_Datetime,
SlsperID,
OriginalLineRef,
BeforeVATPrice,
BeforeVATAmount,
AfterVATPrice,
AfterVATAmount,
VATAmount
from OM_SalesOrdDet
where LUpd_DateTime >= @from
and Crtd_DateTime < @from



In [6]:
# print(sql)

In [42]:
table_name = "sync_dms_sod1"
table_temp = "sync_dms_sod_temp"

In [28]:
#INSERT
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
# bq_values_insert(df, f"{table_temp}", 3)
# sql = \
# f"""
# DROP TABLE IF EXISTS biteam.{table_name};
# CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
# PARTITION BY DATE(crtd_datetime)
# CLUSTER BY branchid,origordernbr,slsperid,lineref
# """
# print(sql)
# execute_bq_query(sql)

dsql = \
f"""
delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow}'
"""
print("delete_sql: ", dsql)
execute_bq_query(dsql)


bq_values_insert(df, f"{table_name}", 2)
# sql = \
# f"""
# DROP TABLE IF EXISTS biteam.{table_temp};
# """
# execute_bq_query(sql)

1it [01:26, 86.78s/it]


In [62]:
#UPDATE
df_update = get_ms_df(usql)
df_update.columns = lower_col(df_update)
print (df_update.columns)
df_update['crtd_datetime'] = df_update.crtd_datetime.dt.normalize()
df_update1 = df_update['crtd_datetime']
df_update1.drop_duplicates(inplace=True)
tpl_dt = tuple(df_update1.dt.strftime('%Y-%m-%d').to_list()) + ('1900-01-01',)
# tpl_dt
df_update1 = df_update['branchid'] + df_update['ordernbr'] + df_update['lineref']
df_update1.drop_duplicates(inplace=True)
tpl_pk = tuple(df_update1.to_list()) + ('',)
del_sql = \
f"""
DELETE FROM biteam.{table_name}
WHERE
DATE(crtd_datetime) in {tpl_dt}
AND concat(branchid,ordernbr,lineref) in {tpl_pk}
"""
print("del_sql ",del_sql)
execute_bq_query(del_sql)
df_update['inserted_at'] = datetime.now()
bq_values_insert(df_update, f"{table_name}", 2)

Index(['branchid', 'ordernbr', 'lineref', 'freeitem', 'invtid', 'lineqty',
       'ordertype', 'origordernbr', 'siteid', 'crtd_prog', 'crtd_user',
       'crtd_datetime', 'lupd_datetime', 'slsperid', 'originallineref',
       'beforevatprice', 'beforevatamount', 'aftervatprice', 'aftervatamount',
       'vatamount'],
      dtype='object')
del_sql  
DELETE FROM biteam.sync_dms_sod1
WHERE
DATE(crtd_datetime) in ('2022-05-12', '1900-01-01')
AND concat(branchid,ordernbr,lineref) in ('MR0001HD0-0522-0401100001', 'MR0001HD0-0522-0401100002', 'MR0001HD0-0522-0401100003', 'MR0001HD0-0522-0401100004', 'MR0001HD0-0522-0401100005', 'MR0001HD0-0522-0401100006', 'MR0001HD0-0522-0401100007', 'MR0001HD0-0522-0401100008', 'MR0001HD0-0522-0401100009', 'MR0001HD0-0522-0401100010', 'MR0001HD0-0522-0401100011', 'MR0001HD0-0522-0401100012', '')



1it [00:05,  5.60s/it]


In [60]:
execute_bq_query(del_sql)

In [61]:
bq_values_insert(df_update, f"{table_name}", 2)

1it [00:04,  4.47s/it]


In [58]:
tpl_pk + ('',)

('MR0001HD0-0522-0401100001',
 'MR0001HD0-0522-0401100002',
 'MR0001HD0-0522-0401100003',
 'MR0001HD0-0522-0401100004',
 'MR0001HD0-0522-0401100005',
 'MR0001HD0-0522-0401100006',
 'MR0001HD0-0522-0401100007',
 'MR0001HD0-0522-0401100008',
 'MR0001HD0-0522-0401100009',
 'MR0001HD0-0522-0401100010',
 'MR0001HD0-0522-0401100011',
 'MR0001HD0-0522-0401100012',
 '')

In [57]:
tpl_dt + ('1900-01-01',)

('2022-05-12', '1900-01-01')

In [56]:
df_update1.dt.strftime('%Y-%m-%d').to_list()

AttributeError: Can only use .dt accessor with datetimelike values

In [2]:
sql = \
"""
SELECT
CONCAT(BranchID, OrderNbr, LineRef) as pk,
DiscAmt,
DocDiscAmt,
GroupDiscAmt1
from OM_SalesOrdDet
"""

In [3]:
df = get_ms_df(sql)

In [4]:
df.head()

,pk,DiscAmt,DocDiscAmt,GroupDiscAmt1
0,MR0016AJ052021-0000300001,0.0,0.0,0.0
1,MR0014AJ120522-0000300001,0.0,0.0,0.0
2,MR0014AJ120522-0000300002,0.0,0.0,0.0
3,MR0014AJ120522-0000600001,0.0,0.0,0.0
4,MR0001CO052021-0000200001,0.0,0.0,0.0


In [5]:
bq_values_insert(df, "sod_added", 1)

In [ ]:
# SELECT a.*, b.*except(pk) FROM `spatial-vision-343005.biteam.sync_dms_sod1` a
# left join `biteam.sod_added` b on a.pk = b.pk

In [10]:
df_update.head()

,branchid,ordernbr,lineref,freeitem,invtid,lineqty,ordertype,origordernbr,siteid,crtd_prog,crtd_user,crtd_datetime,lupd_datetime,slsperid,originallineref,beforevatprice,beforevatamount,aftervatprice,aftervatamount,vatamount


In [19]:
# dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

# insert = PythonOperator(task_id="insert", python_callable=insert, dag=dag)

# update = PythonOperator(task_id="update", python_callable=update, dag=dag)

# dummy_start >> insert >> update